# Plotly Dash Web App

## Install Dependencies

In [1]:
%%sh
pip install dash --upgrade
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_table

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
uploaded = files.upload()

Saving ngrok_api_key.txt to ngrok_api_key.txt


In [3]:
with open('ngrok_api_key.txt', 'r') as f:
    key = f.read()

In [4]:
# import ngrok_api_key
print(key)

2GjxcYtzq6xj9Ubf4t34QQQ2Sh7_9ojiB72BuPpZWBCmoXDg


## Install Ngrok

In [5]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.1M  100 13.1M    0     0  11.8M      0  0:00:01  0:00:01 --:--:-- 11.8M


In [6]:
%%sh
./ngrok authtoken key

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
%%writefile dash_app.py

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)




Writing dash_app.py


In [8]:
# launch ngrok
get_ipython().system_raw('./ngrok http 8050 &')

In [9]:
# # get url with ngrok
# !curl -s http://localhost:4040/api/tunnels -k | python -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [10]:
# !python dash_app.py

In [11]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 8.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=cd0f900108d666b71ecb52a5a6b4c95632c64bd6ae343b051f2122344836fbab
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [12]:
key

'2GjxcYtzq6xj9Ubf4t34QQQ2Sh7_9ojiB72BuPpZWBCmoXDg'

In [13]:
from pyngrok import ngrok, conf

# print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = key
port = 8050
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

 * ngrok tunnel "http://7935-34-82-251-113.ngrok.io" -> "http://127.0.0.1:8050"


In [14]:
# import os
# import threading

# from flask import Flask
# from pyngrok import ngrok

# os.environ["FLASK_ENV"] = "development"

# app = Flask(__name__)
# port = 5000

# # Open a ngrok tunnel to the HTTP server
# public_url = ngrok.connect(port).public_url
# print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# # Update any base URLs to use the public ngrok URL
# app.config["BASE_URL"] = public_url

# # ... Update inbound traffic via APIs to use the public-facing ngrok URL


# # Define Flask routes
# @app.route("/")
# def index():
#     return "Hello from Colab!"

# # Start the Flask server in a new thread
# threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()